In [67]:
import pandas as pd
import re
import numpy as np

import sys
sys.path.append("../")

import src.support as sp

#### Data Wrangling & Data Cleaning

In [68]:
df = pd.read_csv("../data/csv/all_games_raw.csv")

In [69]:
df.head()

,name,platform,release_date,summary,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998","As a young boy, Link is tricked by Ganondorf, ...",99,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",As most major publishers' development efforts ...,98,7.4
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.7
3,SoulCalibur,Dreamcast,"September 8, 1999","This is a tale of souls and swords, transcendi...",98,8.4
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,7.9


In [70]:
df.shape

(18800, 6)

In [71]:
df.info()

# "user_review" column is object, must be float.
# release_date" column ins object, must be date.

# "summary" column has nulls.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18800 entries, 0 to 18799
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          18800 non-null  object
 1   platform      18800 non-null  object
 2   release_date  18800 non-null  object
 3   summary       18686 non-null  object
 4   meta_score    18800 non-null  int64 
 5   user_review   18800 non-null  object
dtypes: int64(1), object(5)
memory usage: 881.4+ KB


In [72]:
df.isnull().sum() * 100/18800

# 0.6% of values in summary column are nulls. Either way, they are irrelevant for the study.

name            0.000000
platform        0.000000
release_date    0.000000
summary         0.606383
meta_score      0.000000
user_review     0.000000
dtype: float64

In [73]:
df["summary"] = df["summary"].fillna("no data")

In [74]:
df.isnull().sum() * 100/18800

name            0.0
platform        0.0
release_date    0.0
summary         0.0
meta_score      0.0
user_review     0.0
dtype: float64

In [75]:
df["user_review"].describe()

count     18800
unique       94
top         tbd
freq       1365
Name: user_review, dtype: object

In [76]:
df["user_review"] = df["user_review"].replace("tbd", np.NaN)

# Now we can change "user_review" to float. We decide to change the NaN with the mean of "user_review" column
df["user_review"] = df["user_review"].astype(dtype="float")
df["user_review"] = df["user_review"].fillna(df["user_review"].mean())

In [77]:
# release_date column to datetime.
df["release_date"] = pd.to_datetime(df["release_date"])

In [78]:
df.dtypes

name                    object
platform                object
release_date    datetime64[ns]
summary                 object
meta_score               int64
user_review            float64
dtype: object

In [79]:
df.duplicated().sum()

# No duplicated rows.

0

In [80]:
df.describe().T

# Everything is apparently correct.

,count,mean,std,min,25%,50%,75%,max
meta_score,18800.0,70.648883,12.225006,20.0,64.0,72.0,80.0,99.0
user_review,18800.0,6.990846,1.301561,0.2,6.4,7.2,7.9,9.7


In [81]:
df.describe(include="object").T

# Column "name" is repeated. A game may be in different platforms.

,count,unique,top,freq
name,18800,12254,Cars,9
platform,18800,22,PC,4864
summary,18800,14648,no data,114


In [87]:
# Groupby name and platform to get the mean of games duplicated by "name" and "platform"

df2 = df.groupby(["name", "platform"]).mean().reset_index()
df2.head(2)

C:\Users\jorge\AppData\Local\Temp\ipykernel_14792\3139701434.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df2 = df.groupby(["name", "platform"]).mean().reset_index()


,name,platform,meta_score,user_review
0,#DRIVE,Switch,69.0,6.990846
1,#IDARB,Xbox One,77.0,6.900000


In [89]:
# This column has spaces, that's why we need to use strip method.
df2["platform"] = df2["platform"].str.strip()

In [90]:
# The other row has dissapeared.
df2[(df2["name"] == "Cars") & (df2["platform"] == "PlayStation 2")]

,name,platform,meta_score,user_review
2324,Cars,PlayStation 2,62.5,5.75


In [91]:
df2[df2["name"] == "Cars"]

# Checking everything has worked out.

,name,platform,meta_score,user_review
2321,Cars,GameCube,71.0,7.20
2322,Cars,PC,73.0,7.70
2323,Cars,PSP,70.0,6.90
2324,Cars,PlayStation 2,62.5,5.75
2325,Cars,Wii,65.0,7.00
2326,Cars,Xbox,70.0,8.00
2327,Cars,Xbox 360,57.5,6.10


In [92]:
df2.shape

(18690, 4)

In [93]:
# From df we have to eliminate all duplicates by "name" & "platform" so we can keep one element and add
# "meta_score" and "user_review" to this element
dfn = df.drop_duplicates(subset=["name", "platform"], keep="first")
dfn.shape

(18690, 6)

In [94]:
# removing blanks.

dfn["platform"] = dfn["platform"].str.strip()

C:\Users\jorge\AppData\Local\Temp\ipykernel_14792\3306062980.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfn["platform"] = dfn["platform"].str.strip()


In [95]:
# There is only one game for Cars & Playstation 2 (before we had 2)

dfn[(dfn["name"] == "Cars") & (dfn["platform"] == "PlayStation 2")]

,name,platform,release_date,summary,meta_score,user_review
10234,Cars,PlayStation 2,2006-06-06,Race to the finish line as you live all the fu...,71,8.7


In [96]:
# Creating an id so we can merge both df (df3 and dfn)
df3 = df2.sort_values(by= ["name"])
df3["id"] = df3["name"]+df3["platform"]
df3.head(1)

,name,platform,meta_score,user_review,id
0,#DRIVE,Switch,69.0,6.990846,#DRIVESwitch


In [97]:
dfn = dfn.sort_values(by= ["name"])
dfn["id"] = dfn["name"]+dfn["platform"]
dfn.head(1)

,name,platform,release_date,summary,meta_score,user_review,id
11682,#DRIVE,Switch,2021-02-16,#DRIVE is an endless driving videogame inspire...,69,6.990846,#DRIVESwitch


In [98]:
dropcolumns = ["meta_score", "user_review", "name", "platform"]
dfn.drop(columns=dropcolumns, inplace=True)
dfn.head(1)

,release_date,summary,id
11682,2021-02-16,#DRIVE is an endless driving videogame inspire...,#DRIVESwitch


In [99]:
# Merging both df so we can keep the mean for games that are duplicated.
df_final = df3.merge(dfn, how='inner', on="id")
df_final.drop(["id"], axis=1, inplace=True)
df_final.head(1)

,name,platform,meta_score,user_review,release_date,summary
0,#DRIVE,Switch,69.0,6.990846,2021-02-16,#DRIVE is an endless driving videogame inspire...


In [100]:
# Deleting blanks.
df["platform"] = df["platform"].str.strip()
# Comparing df with df_final
df[(df["name"] == "Cars") & (df["platform"] == "PlayStation 2")]

,name,platform,release_date,summary,meta_score,user_review
10234,Cars,PlayStation 2,2006-06-06,Race to the finish line as you live all the fu...,71,8.7
16947,Cars,PlayStation 2,2006-06-06,Race to the finish line as you live all the fu...,54,2.8


In [101]:
df_final["platform"] = df_final["platform"].str.strip()
df_final[(df_final["name"] == "Cars") & (df_final["platform"] == "PlayStation 2")]

# "meta_score" and "user_review" have the mean of both games.

,name,platform,meta_score,user_review,release_date,summary
2323,Cars,PlayStation 2,62.5,5.75,2006-06-06,Race to the finish line as you live all the fu...


In [102]:
df_final.shape

(18690, 6)

In [103]:
df_final.isnull().sum()

name            0
platform        0
meta_score      0
user_review     0
release_date    0
summary         0
dtype: int64

In [104]:
df_final.dtypes

name                    object
platform                object
meta_score             float64
user_review            float64
release_date    datetime64[ns]
summary                 object
dtype: object

In [106]:
df_final = df_final[["name", "platform", "release_date", "summary", "meta_score", "user_review"]]
df_final.head(1)

,name,platform,release_date,summary,meta_score,user_review
0,#DRIVE,Switch,2021-02-16,#DRIVE is an endless driving videogame inspire...,69.0,6.990846


In [35]:
df_final.to_csv("../data/csv/all_games.csv")

In [36]:
# We need to create a new dataframe so we have a CSV with unique videogames that will connect with all tables
# in MySQL

df = pd.read_csv("../data/csv/all_games.csv")
df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [37]:
df_unique = df.groupby(by="name").mean().reset_index()
df_unique.head(1)

C:\Users\jorge\AppData\Local\Temp\ipykernel_14792\3356141005.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_unique = df.groupby(by="name").mean().reset_index()


,name,meta_score,user_review
0,#DRIVE,69.0,6.990846


In [38]:
df = df.drop_duplicates(subset="name", keep="first")
df.drop(["summary", "platform", "meta_score", "user_review"], axis=1, inplace=True)
df.head(1)

,name,release_date
0,#DRIVE,2021-02-16


In [39]:
df_unique = df_unique.merge(df, how="inner", on="name")
df_unique.head(1)

,name,meta_score,user_review,release_date
0,#DRIVE,69.0,6.990846,2021-02-16


In [40]:
df_unique.to_csv("../data/csv/all_games_unique.csv")